# Train Notebook

In [1]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import os
import cv2
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

if "notebooks" in os.getcwd():
    print("Running in Jupyter Notebook")


COLORS = [
    (0, 0, 0), (67, 161, 255), (167, 146, 11), (136, 126, 185), (44, 52, 10), (25, 33, 189), (73, 197, 184),
    (20, 165, 16), (48, 37, 106), (98, 213, 120), (21, 104, 190), (191, 106, 197), (142, 63, 109), (155, 22, 122),
    (43, 152, 125), (128, 89, 85), (11, 1, 133), (126, 45, 174), (32, 111, 29), (55, 31, 198), (70, 250, 116),
    (216, 21, 138), (100, 0, 176), (171, 236, 47), (193, 137, 224), (36, 152, 214), (154, 165, 67), (73, 8, 110),
    (67, 161, 255), (167, 146, 11), (136, 126, 185), (44, 52, 10), (25, 33, 189), (73, 197, 184), (20, 165, 16),
    (48, 37, 106), (98, 213, 120), (21, 104, 190), (191, 106, 197), (142, 63, 109), (155, 22, 122), (43, 152, 125),
    (128, 89, 85), (11, 1, 133), (126, 45, 174), (32, 111, 29), (55, 31, 198), (70, 250, 116), (216, 21, 138)
]

In [2]:
dataDir='data'
dataType='train2020'
annFile='{}/annotations/instances_attributes_{}.json'.format(dataDir,dataType)
maskFolder='{}/annotations/{}/'.format(dataDir,dataType)
imgFolder='{}/images/{}/'.format(dataDir,dataType)

# initialize COCO api for instance annotations
db=COCO(annFile)
d_cats = {x['id']: x['name'] for x in db.dataset['categories']}

loading annotations into memory...
Done (t=3.48s)
creating index...
index created!


In [3]:

def get_image(img_id):
    img = db.loadImgs(img_id)[0]
    I = io.imread('{}/images/{}/{}'.format(dataDir,dataType,img['file_name']))
    return I

def get_mask(img_id):
    file_name = db.loadImgs(img_id)[0]['file_name']
    mask = cv2.imread(f"{maskFolder}/{file_name.split('.')[0]}_seg.png", cv2.IMREAD_GRAYSCALE)
    return mask

def get_annotations(img_id):
    annIds = db.getAnnIds(imgIds=img_id)
    anns = db.loadAnns(annIds)
    return anns

def box_label(image, box, mask=None, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
  if mask is not None:
    image = image * (1-mask) + (image + np.array([[list(color)]])) * mask / 2
  lw = max(round(sum(image.shape) / 2 * 0.003), 2)
  p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
  image = cv2.rectangle(image.astype(np.uint8), p1, p2, color, thickness=lw, lineType=cv2.LINE_AA)
  if label:
    tf = max(lw - 1, 1)  # font thickness
    w, h = cv2.getTextSize(label, 0, fontScale=lw / 3, thickness=tf)[0]  # text width, height
    outside = p1[1] - h >= 3
    p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
    image = cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)  # filled
    image = cv2.putText(image,
                label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                0,
                lw / 3,
                txt_color,
                thickness=tf,
                lineType=cv2.LINE_AA)
    return image
  
def plot_boxes(img_id, ax=None):
    img = get_image(img_id)
    anns = get_annotations(img_id)

    plt.imshow(img)
    for ann in anns:
        color = COLORS[ann['category_id']]
        bbox = ann['bbox']
        bbox = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]
        img = box_label(img, bbox, label=db.loadCats(ann['category_id'])[0]['name'], color=color)

    if ax is not None:
        ax.imshow(img)
        ax.axis('off')
    else:
        plt.imshow(img)
        plt.axis('off')

def plot_mask(mask_id):
    mask = cv2.imread(f"{maskFolder}/{mask_id}", cv2.IMREAD_GRAYSCALE)
    plt.imshow(mask, cmap='gray')
    plt.axis('off')
    # plt.show()

def overlay_mask(img_file, alpha=0.7, ax=None):
    mask = cv2.imread(f"{maskFolder}/{img_file.split('.')[0]}_seg.png", cv2.IMREAD_GRAYSCALE)
    img = plt.imread(f"{imgFolder}/{img_file}", cv2.IMREAD_COLOR)

    # For all the values in the mask, we will create a new mask with the same shape but with the color of the category
    mask_color = np.zeros((mask.shape[0], mask.shape[1], 3))
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            mask_color[i, j] = COLORS[mask[i, j]]

    mask_color = mask_color.astype(np.uint8)

    # Combine original image with colored mask
    image = cv2.addWeighted(img, 1 - alpha, mask_color, alpha, 0)

    # image = cv2.addWeighted(img, 1, mask_color, 1, 0)

    if ax is not None:
        ax.imshow(image)
        ax.axis('off')
    else:
        # plt.figure(figsize=(10, 10))
        plt.imshow(image)
        plt.axis('off')
  

In [4]:
from ultralytics import YOLO

# Load a model
# "-seg" is added to the end of "yolov8X" for instance segmentation
model = YOLO("yolov8n-seg.yaml")  # build a new model from scratch
model = YOLO("yolov8n-seg.pt")  # load a pretrained model (recommended for training)

In [5]:
model

YOLO(
  (model): SegmentationModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_runnin

In [6]:
ids = db.getImgIds()

In [8]:
if "mmsegmentation" in os.getcwd():
    os.chdir("..")
    
image = get_image(ids[0])
results = model.predict(image)  # predict on an image
boxes = results[0].boxes.data.numpy()
boxes


0: 640x640 2 persons, 64.1ms
Speed: 2.4ms preprocess, 64.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


array([[     44.043,      1.0276,      585.03,      997.56,      0.7854,           0],
       [     473.16,     0.21807,      626.75,      274.43,     0.38667,           0]], dtype=float32)

In [11]:
results[0]

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: ultralytics.engine.results.Masks object
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair

In [13]:
def get_image_masks_centers_boxes(url, model):
    # Get image
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))

    # Perform prediction
    results = model(image)

    # Get the center of each mask
    masks = results[0].masks.data.numpy()
    centers = []

    for mask in masks:
        mask = cv2.resize(mask, (image.size[0], image.size[1]))
        mask = mask > 0.5
        mask = mask.astype(np.uint8)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(contours) > 0:
            M = cv2.moments(contours[0])
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            centers.append(center)
        else:
            centers.append(None)

    return np.array(image), masks, np.array(centers), results[0].boxes.data.numpy()

    
def plot_bboxes(image, boxes, masks=None , labels=[], colors=[], score=True, conf=None):
  #Define COCO Labels
  if labels == []:
    labels = {0: u'__background__', 1: u'person', 2: u'bicycle',3: u'car', 4: u'motorcycle', 5: u'airplane', 6: u'bus', 7: u'train', 8: u'truck', 9: u'boat', 10: u'traffic light', 11: u'fire hydrant', 12: u'stop sign', 13: u'parking meter', 14: u'bench', 15: u'bird', 16: u'cat', 17: u'dog', 18: u'horse', 19: u'sheep', 20: u'cow', 21: u'elephant', 22: u'bear', 23: u'zebra', 24: u'giraffe', 25: u'backpack', 26: u'umbrella', 27: u'handbag', 28: u'tie', 29: u'suitcase', 30: u'frisbee', 31: u'skis', 32: u'snowboard', 33: u'sports ball', 34: u'kite', 35: u'baseball bat', 36: u'baseball glove', 37: u'skateboard', 38: u'surfboard', 39: u'tennis racket', 40: u'bottle', 41: u'wine glass', 42: u'cup', 43: u'fork', 44: u'knife', 45: u'spoon', 46: u'bowl', 47: u'banana', 48: u'apple', 49: u'sandwich', 50: u'orange', 51: u'broccoli', 52: u'carrot', 53: u'hot dog', 54: u'pizza', 55: u'donut', 56: u'cake', 57: u'chair', 58: u'couch', 59: u'potted plant', 60: u'bed', 61: u'dining table', 62: u'toilet', 63: u'tv', 64: u'laptop', 65: u'mouse', 66: u'remote', 67: u'keyboard', 68: u'cell phone', 69: u'microwave', 70: u'oven', 71: u'toaster', 72: u'sink', 73: u'refrigerator', 74: u'book', 75: u'clock', 76: u'vase', 77: u'scissors', 78: u'teddy bear', 79: u'hair drier', 80: u'toothbrush'}
  #Define colors
  if colors == []:
    #colors = [(6, 112, 83), (253, 246, 160), (40, 132, 70), (205, 97, 162), (149, 196, 30), (106, 19, 161), (127, 175, 225), (115, 133, 176), (83, 156, 8), (182, 29, 77), (180, 11, 251), (31, 12, 123), (23, 6, 115), (167, 34, 31), (176, 216, 69), (110, 229, 222), (72, 183, 159), (90, 168, 209), (195, 4, 209), (135, 236, 21), (62, 209, 199), (87, 1, 70), (75, 40, 168), (121, 90, 126), (11, 86, 86), (40, 218, 53), (234, 76, 20), (129, 174, 192), (13, 18, 254), (45, 183, 149), (77, 234, 120), (182, 83, 207), (172, 138, 252), (201, 7, 159), (147, 240, 17), (134, 19, 233), (202, 61, 206), (177, 253, 26), (10, 139, 17), (130, 148, 106), (174, 197, 128), (106, 59, 168), (124, 180, 83), (78, 169, 4), (26, 79, 176), (185, 149, 150), (165, 253, 206), (220, 87, 0), (72, 22, 226), (64, 174, 4), (245, 131, 96), (35, 217, 142), (89, 86, 32), (80, 56, 196), (222, 136, 159), (145, 6, 219), (143, 132, 162), (175, 97, 221), (72, 3, 79), (196, 184, 237), (18, 210, 116), (8, 185, 81), (99, 181, 254), (9, 127, 123), (140, 94, 215), (39, 229, 121), (230, 51, 96), (84, 225, 33), (218, 202, 139), (129, 223, 182), (167, 46, 157), (15, 252, 5), (128, 103, 203), (197, 223, 199), (19, 238, 181), (64, 142, 167), (12, 203, 242), (69, 21, 41), (177, 184, 2), (35, 97, 56), (241, 22, 161)]
    colors = [(89, 161, 197),(67, 161, 255),(19, 222, 24),(186, 55, 2),(167, 146, 11),(190, 76, 98),(130, 172, 179),(115, 209, 128),(204, 79, 135),(136, 126, 185),(209, 213, 45),(44, 52, 10),(101, 158, 121),(179, 124, 12),(25, 33, 189),(45, 115, 11),(73, 197, 184),(62, 225, 221),(32, 46, 52),(20, 165, 16),(54, 15, 57),(12, 150, 9),(10, 46, 99),(94, 89, 46),(48, 37, 106),(42, 10, 96),(7, 164, 128),(98, 213, 120),(40, 5, 219),(54, 25, 150),(251, 74, 172),(0, 236, 196),(21, 104, 190),(226, 74, 232),(120, 67, 25),(191, 106, 197),(8, 15, 134),(21, 2, 1),(142, 63, 109),(133, 148, 146),(187, 77, 253),(155, 22, 122),(218, 130, 77),(164, 102, 79),(43, 152, 125),(185, 124, 151),(95, 159, 238),(128, 89, 85),(228, 6, 60),(6, 41, 210),(11, 1, 133),(30, 96, 58),(230, 136, 109),(126, 45, 174),(164, 63, 165),(32, 111, 29),(232, 40, 70),(55, 31, 198),(148, 211, 129),(10, 186, 211),(181, 201, 94),(55, 35, 92),(129, 140, 233),(70, 250, 116),(61, 209, 152),(216, 21, 138),(100, 0, 176),(3, 42, 70),(151, 13, 44),(216, 102, 88),(125, 216, 93),(171, 236, 47),(253, 127, 103),(205, 137, 244),(193, 137, 224),(36, 152, 214),(17, 50, 238),(154, 165, 67),(114, 129, 60),(119, 24, 48),(73, 8, 110)]
  
  #plot each boxes (and mask)

  for i in range(boxes.shape[0]):
    box = boxes[i]
    if masks is not None:
      mask = masks[i]
    else:
      mask = None

    #add score in label if score=True
    if score :
      label = labels[int(box[-1])+1] + " " + str(round(100 * float(box[-2]),1)) + "%"
    else :
      label = labels[int(box[-1])+1]
    #filter every box under conf threshold if conf threshold setted
    if conf :
      if box[-2] > conf:
        color = colors[int(box[-1])]
        image = box_label(image, box, mask, label, color)
    else:
      color = colors[int(box[-1])]
      image = box_label(image, box, mask, label, color)

  #show image
  # image = cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_BGR2RGB)

  try:
    import google.colab
    IN_COLAB = True
  except:
    IN_COLAB = False

  if IN_COLAB:
    cv2_imshow(image) #if used in Colab
  else :
    plt.imshow(image) #if used in Python

# def get_image(url):
#   response = requests.get(url)
#   image = Image.open(BytesIO(response.content))
#   return np.asarray(image)

from ultralytics import YOLO
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import cv2

# Load a model
# "-seg" is added to the end of "yolov8X" for instance segmentation
model = YOLO("yolov8n-seg.yaml")  # build a new model from scratch
model = YOLO("yolov8n-seg.pt")  # load a pretrained model (recommended for training)

model = YOLO("yolov8n-seg.yaml")  # build a new model from scratch
model = YOLO("yolov8n-seg.pt")  # load a pretrained model (recommended for training)

image, masks, centers, boxes = get_image_masks_centers_boxes(ids[0], model=model)
# fig, ax = plt.subplots(1, 3, figsize=(15, 6))
# ax[0].imshow(cv2.resize(np.max(masks, axis=0), (image.shape[1], image.shape[0])))
# ax[0].plot(centers[:, 0], centers[:, 1], 'ro')
# ax[1].imshow(image)
masks = cv2.resize(masks.transpose((1,2,0)), (image.shape[1],image.shape[0])).transpose((2,0,1))
plot_bboxes(image, boxes, masks=masks[..., np.newaxis], conf=0.8)
plt.show()

MissingSchema: Invalid URL '16145': No scheme supplied. Perhaps you meant https://16145?

In [6]:
## Terminal
# !git clone -b main https://github.com/open-mmlab/mmsegmentation.git
# !cd mmsegmentation
# !pip install -v -e .

# Download models
# Install requirements
# !mim download mmsegmentation --config pspnet_r50-d8_4xb2-40k_cityscapes-512x1024 --dest .

# !pip3 install openmim
# !mim install mmengine
# !mim install "mmcv>=2.0.0"

In [7]:
# import os
# if "mmsegmentation" not in os.getcwd():
#     os.chdir('mmsegmentation')
    
# from mmseg.apis import inference_model, init_model, show_result_pyplot
# import mmcv

# config_file = 'pspnet_r50-d8_4xb2-40k_cityscapes-512x1024.py'
# checkpoint_file = 'pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'

# # build the model from a config file and a checkpoint file
# model = init_model(config_file, checkpoint_file, device='cuda:0')

# # test a single image and show the results
# img = 'demo/demo.png'  # or img = mmcv.imread(img), which will only load it once
# result = inference_model(model, img)
# # visualize the results in a new window
# show_result_pyplot(model, img, result, show=True)
# # or save the visualization results to image files
# # you can change the opacity of the painted segmentation map in (0, 1].
# show_result_pyplot(model, img, result, show=True, out_file='result.jpg', opacity=0.5)
# # test a video and show the results
# video = mmcv.VideoReader('video.mp4')
# for frame in video:
#    result = inference_model(model, frame)
#    show_result_pyplot(model, frame, result, wait_time=1)

ModuleNotFoundError: No module named 'mmcv'